In [1]:
#import the necessary packages
import numpy as np
import pandas as pd

In [6]:
# open the .csv file - the "header=0" part sets the first row as header of the dataframe
df = pd.read_csv('book_list.csv', header=0)
for i in df.columns:
  print("column name: ", i)
# now, I don't want to show the last column, i.e. Unnamed:10 --> I will delete it
df = df.drop(df.columns[-1], axis=1)

column name:  title
column name:  # pages
column name:  year of publication
column name:  author
column name:  publisher
column name:  year of birth
column name:  year of death
column name:  nationality
column name:  sex
column name:  language
column name:  Unnamed: 10


In [7]:
df

,title,# pages,year of publication,author,publisher,year of birth,year of death,nationality,sex,language
0,la valle oscura,309,2020,anna wiener,adelphi,1987,/,USA,f,italiano
1,gioco all'alba,152,1961,arthur schnitzler,adelphi,1862,1931,AUSTRIA,m,italiano
2,il ritorno di casanova,149,1900,arthur schnitzler,adelphi,1862,1931,AUSTRIA,m,italiano
3,helgoland,227,2020,carlo rovelli,adelphi,1956,/,ITALY,m,italiano
4,"godel, escher, bach",852,1979,douglas r. hofstadter,adelphi,1977,/,USA,m,italiano
...,...,...,...,...,...,...,...,...,...,...
452,sociologia della religione - induismo e buddhismo,327,1916,max weber,newton compton,1864,1920,GERMANY,m,italiano
453,storia d'europa dalle invasioni al XVI secolo,441,1936,hernri pirenne,newton compton,1862,1935,BELGIUM,m,italiano
454,the longest journey,327,1907,edward morgan forster,new directions,1879,1970,ENGLISH,m,english
455,macbeth,190,1606,william shakespeare,Thomson Learning,/,1616,ENGLAND,m,english


In [8]:
# morevoer, I want the row index to start from 1 and not from 0
df = df.reset_index(drop=True)
df.index += 1
df

,title,# pages,year of publication,author,publisher,year of birth,year of death,nationality,sex,language
1,la valle oscura,309,2020,anna wiener,adelphi,1987,/,USA,f,italiano
2,gioco all'alba,152,1961,arthur schnitzler,adelphi,1862,1931,AUSTRIA,m,italiano
3,il ritorno di casanova,149,1900,arthur schnitzler,adelphi,1862,1931,AUSTRIA,m,italiano
4,helgoland,227,2020,carlo rovelli,adelphi,1956,/,ITALY,m,italiano
5,"godel, escher, bach",852,1979,douglas r. hofstadter,adelphi,1977,/,USA,m,italiano
...,...,...,...,...,...,...,...,...,...,...
453,sociologia della religione - induismo e buddhismo,327,1916,max weber,newton compton,1864,1920,GERMANY,m,italiano
454,storia d'europa dalle invasioni al XVI secolo,441,1936,hernri pirenne,newton compton,1862,1935,BELGIUM,m,italiano
455,the longest journey,327,1907,edward morgan forster,new directions,1879,1970,ENGLISH,m,english
456,macbeth,190,1606,william shakespeare,Thomson Learning,/,1616,ENGLAND,m,english
